In [1]:
from pathlib import Path

import Metashape

from result import Ok, Err, Result

from benthoscan.registration import PointCloud, PointCloudLoader
from benthoscan.backends import metashape as backend
from benthoscan.utils.log import logger

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
PATHS: dict[str, Path] = {
    "DOCUMENT_IN": Path(
        "/media/martin/pcie_01/acfr_revisits_metashape_projects/qd61g27j_working_version.psz"
    ),
    "DOCUMENT_OUT": Path(
        "/media/martin/pcie_01/acfr_revisits_metashape_projects_test/qd61g27j_project_saved.psz"
    ),
    "CACHE": Path("/home/martin/dev/benthoscan/.cache/"),
}

backend.log_internal_data()

result: Result[str, str] = backend.load_project(PATHS.get("DOCUMENT_IN"))
match result:
    case Ok(message):
        logger.info(message)
    case Err(message):
        logger.error(message)

backend.log_internal_data()

2024-08-05 15:30:00.590 | INFO     | benthoscan.backends.metashape.context:log_internal_data:34 - {'document': None}


LoadProject: path = /media/martin/pcie_01/acfr_revisits_metashape_projects/qd61g27j_working_version.psz


2024-08-05 15:32:53.976 | INFO     | __main__:<module>:16 - loaded document /media/martin/pcie_01/acfr_revisits_metashape_projects/qd61g27j_working_version.psz successfully
2024-08-05 15:32:53.978 | INFO     | benthoscan.backends.metashape.context:log_internal_data:34 - {'document': <Document '/media/martin/pcie_01/acfr_revisits_metashape_projects/qd61g27j_working_version.psz'>}


loaded project in 173.291 sec


### Export dense models from the backend

In [ ]:
test = Metashape.app.document

logger.info(test)
logger.info(backend.context._backend_data)

PointCloudLoaders = dict[int, PointCloudLoader]

result: Result[PointCloudLoaders, str] = backend.request_dense_models(
    PATHS["CACHE"], overwrite=True
)

In [ ]:
match result:
    case Ok(loaders):
        for key, loader in loaders.items():
            logger.info(f"{key}: {loader}")
    case Err(message):
        logger.error(message)

for key, loader in loaders.items():
    pc: PointCloud = loader()
    logger.info(pc)